In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade  textblob gensim pytorch-nlp swifter


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import itertools
import sys
from textblob import TextBlob, Word
import numpy as np
import random
import re
import swifter
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import os
import pandas as pd
import gensim
import warnings
import nltk

max_length = 100
# Hyperparameters
embedding_dim = 100  # embedding dimension
hidden_dim = 100  # LSTM hidden dimensions
num_layers = 1  # number of LSTM layers
batch_size = 64  # batch size
num_epochs = 10  # number of epochs to train
lr = 0.001  # learning rate


def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


In [ ]:
%%writefile get_data.sh
if [ ! -f ner_dataset.csv ]; then
  wget -O ner_dataset.csv https://www.dropbox.com/s/mbfv0x988mdj89h/ner_dataset.csv?dl=0
fi


In [ ]:
!bash get_data.sh

In [ ]:
data= pd.read_csv("./ner_dataset.csv",encoding="latin1")
data = data.fillna(method='ffill')
data.head()

In [ ]:
print("Unique Words in corpus:",data['Word'].nunique())
print("Unique Tag in corpus:",data['Tag'].nunique())

In [ ]:
words = list(set(data['Word'].values))
words.append("ENDPAD")
num_words = len(words)
tags = list(set(data['Tag'].values))
num_tags = len(tags)

In [ ]:
class SentenceGetter(object):
  def __init__(self,data):
    self.n_sent = 1 #counter
    self.data = data
    agg_func = lambda s:[(w,p,t) for w,p,t in zip(s['Word'].tolist(),s['POS'].tolist(),s['Tag'].tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]



getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

In [ ]:
word2idx =  {w : i+1 for i,w in enumerate(words)}
tag2idx  =  {t : i for i,t in enumerate(tags)}

In [ ]:
tokenized_sentences = [[word2idx[w[0]] for w in s]for s in sentences]

tokenized_sentences[0]

In [ ]:
maximum_length = max([len(x) for x in tokenized_sentences])
maximum_length

In [ ]:
pre_X = [F.pad(torch.tensor(x), (0, maximum_length-len(x)), "constant", 0) for x in tokenized_sentences]
X = torch.stack(pre_X)
X.shape

In [ ]:
tokenizer_entities = [[tag2idx[w[2]] for w in s]for s in sentences]
tokenizer_entities[0]

In [ ]:
maximum_tag_length = max([len(y) for y in tokenizer_entities])
maximum_tag_length

In [ ]:
F.pad(torch.tensor(tokenizer_entities[0]), (0, maximum_tag_length-len(tokenizer_entities[0])), "constant", tag2idx["O"])

In [ ]:
pre_y = [F.pad(torch.tensor(tokenizer_entities[i]), (0, maximum_tag_length-len(tokenizer_entities[i])), "constant", tag2idx["O"]) for i in range(X.shape[0])]
y = torch.stack(pre_y)
y.shape

In [ ]:
y_categorical = torch.stack([F.one_hot(y[0], num_classes=num_tags) for i in range(y.shape[0])])
y_categorical.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y_categorical, test_size=0.1, random_state=42)

In [ ]:
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, drop_last=True)


In [ ]:
# Define a BiLSTM model for NER
class BiLSTMForNER(nn.Module):
    def __init__(self, num_words, num_tags, embedding_dim, hidden_dim=100, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # Embedding layer that converts input words to embeddings
        self.word_embeddings = None

        # The LSTM takes word embeddings as inputs, and outputs hidden states with dimensionality hidden_dim //2.
        # It will be bidirectional, meaning one LSTM for the forward pass and one for the backward pass.
        self.lstm = None

        # The linear layer maps from hidden state space to tag space
        self.hidden2tag = None

    def forward(self, sentence, hidden):
        # Get the embeddings for the sentence
        embeds = None
        # Pass the embeddings through the LSTM; lstm_out shape is (len(sentence), batch_size, hidden_dim)
        lstm_out, hidden = None
        # Pass the LSTM output through the linear layer to get the tag space
        tag_space = None
        # Convert the tag space to tag scores, which are log probabilities of the tags
        tag_scores = None
        return tag_scores, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (
            weight.new(2*self.num_layers, batch_size, self.hidden_dim // 2).zero_().to(device),
            weight.new(2*self.num_layers, batch_size, self.hidden_dim // 2).zero_().to(device)
        )
        return hidden

In [ ]:
model = BiLSTMForNER(num_words, num_tags, embedding_dim, hidden_dim, num_layers).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [ ]:
def kl_divergence_loss(outputs, targets, tagset_size):
    """
    Calculate the KL Divergence loss for the outputs with respect to the targets
    """
    # Flatten outputs and targets to compute the distribution loss across all batches and classes
    outputs = outputs.view(-1, tagset_size)
    targets = targets.view(-1, tagset_size)
    # Compute KL Divergence
    kl_loss = F.kl_div(F.log_softmax(outputs, dim=1), F.softmax(targets.to(float), dim=1), reduction='batchmean')
    return kl_loss

In [ ]:
def train_model(model, train_dataloader, optimizer, num_tags, learning_rate, epochs):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        hidden = model.init_hidden(batch_size)
        total_loss = 0
        for sentences, tag_indices in train_dataloader:
            sentences = sentences.to(device)
            tag_indices = tag_indices.to(device)
            # Clear the gradients before each instance
            optimizer.zero_grad()
            # Forward pass
            outputs, hidden = model(sentences, hidden)
            loss = kl_divergence_loss(outputs, tag_indices, num_tags)
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            for hidden_state in hidden:
              hidden_state.detach_()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader)}")


In [ ]:
train_model(model, train_dl, optimizer, num_tags, learning_rate=lr, epochs=num_epochs)

In [ ]:
ix2tag = {ix: tag for tag, ix in tag2idx.items()}

In [ ]:
def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

In [ ]:
from sklearn.metrics import classification_report
def evaluate_model(model, test_dataloader, ix_to_tag):
    model.eval()  # Set the model to evaluation mode
    true_tags = []
    pred_tags = []
    hidden = model.init_hidden(batch_size)
    with torch.no_grad():
        for sentence, tags in test_dataloader:
            sentence = sentence.to(device)
            tags = tags.to(device)
            # Forward pass
            tag_scores, hidden = model(sentence, hidden)
            # Get the predicted tags
            _, max_indices = torch.max(F.softmax(tag_scores, dim=1), dim=-1, keepdim=True)
            one_hot = torch.zeros_like(F.softmax(tag_scores, dim=1))
            one_hot.scatter_(-1, max_indices, 1)
            # Update lists of true tags and predicted tags
            true_tags.extend(tags.tolist())
            pred_tags.extend(one_hot.tolist())
    # Convert index sequences to tag name sequences


    true_tag_names = [ix_to_tag[argmax(ix)] for ix in true_tags]
    pred_tag_names = [ix_to_tag[argmax(ix)] if argmax(ix) < 17 else ix_to_tag[16] for ix in pred_tags ]

    # Calculate and print the classification report
    print(classification_report(true_tag_names, pred_tag_names))

In [ ]:
evaluate_model(model, test_dl, ix2tag)